<a href="https://colab.research.google.com/github/rpdieego/Reinforcement_Learning/blob/master/Reinforcement_Learning_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid World

#### Enviroment

In [0]:
### Set of all possible actions in the grid world environment
# U => Up
# D => Down
# L => Left
# R => Right
ACTION_SPACE = ('U', 'D', 'L', 'R')

In [0]:
#grid world class

class Grid: # Environment
  def __init__ (self, rows, cols, start):
    self.rows = rows
    self.cols = cols
    self.i = start[0]
    self.j = start[1]

  def set(self, rewards, actions):
    # rewards is a dict of: (i,j): r
    # actions is a dict of: (i,j): A
    self.rewards = rewards
    self.actions = actions

  def set_state(self,s):
    #force state
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    #return the current state
    return (self.i, self.j)
  
  def is_terminal(self, s):
    #if state is not listed in the actions dictionary, it means it's a terminal state
    return s not in self.actions

  def get_next_state(self, s, a):
    i, j = s[0], s[1]
    if a in self.actions[(i,j)]:
      if a == 'U':
        i -= 1
      elif a =='D':
        i += 1
      elif a == 'R':
        j += 1
      elif a == 'L':
        j -= 1
    return i,j

  def move(self, action):
    # check if legal move first
    if action in self.actions[(self.i, self.j)]:
      if action == 'U':
        self.i -= 1
      elif action == 'D':
        self.i += 1
      elif action == 'R':
        self.j += 1
      elif action == 'L':
        self.j -= 1
    return self.rewards.get((self.i, self.j), 0)

  def undo_move(self, action):
    if action == 'U':
      self.i += 1
    elif action == 'D':
      self.i -= 1
    elif action == 'R':
      self.j -= 1
    elif action == 'L':
      self.j += 1
    #should never happen
    assert(self.current_state() in self.all_states())
  
  def game_over(self):
    #true if in a state where no actions are possible
    return (self.i,self.j) not in self.actions

  def all_states(self):
    #either a position that has possible next actions or a positions that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())

  

In [0]:
# define grid environment

def standard_grid():
  g = Grid(3, 4, (2,0))
  rewards = {(0,3): 1, (1,3): -1}
  actions = {
      (0,0): ('D','R'),
      (0,1): ('L','R'),
      (0,2): ('L','D','R'),
      (1,0): ('U','D'),
      (1,2): ('U', 'D', 'R'),
      (2,0): ('U','R'),
      (2,1): ('L','R'),
      (2,2): ('L','R','U'),
      (2,3): ('L','U')
  }
  g.set(rewards, actions)
  return g

#### Iterative Policy Evaluation

In [0]:
# import relevant libraries
import numpy as np

In [0]:
# threshold for convergence
conv_threshold = 1e-3

In [0]:
# auxiliar function to print values
def print_values(V,g):
  for i in range(g.rows):
    print('-------------------------------')
    for j in range(g.cols):
      v = V.get((i,j),0)
      if v>= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # - sign take up an extra space
    print('')

In [0]:
# auxiliar function to print policy
def print_policy(P,g):
  print('Policy \n')
  for i in range(g.rows):
    print('-----------------------------')
    for j in range(g.cols):
      a = P.get((i,j),' ')
      print("  %s  |" % a, end='')
    print('')
  print('\n')


In [26]:
# main section

### define transition probabilities dictionary
# s = current state
# a = action
# s' = next state
# key is (s, a, s'), the value is the probability
# that is: transition_probs[(s,a,s')] = p(s' | s,a)

transition_probs = {}

### define the rewards dictionary
# in order to reduce dimensionality of the dictionary, we'll use deterministic
# rewards, r(s, a, s')
rewards = {}

#define grid world environment
grid = standard_grid()

# populate transition_probs and rewards dictionary
for i in range(grid.rows):
  for j in range(grid.cols):
    s = (i,j)
    if not grid.is_terminal(s):
      for a in ACTION_SPACE:
        s2 = grid.get_next_state(s,a)
        transition_probs[(s,a,s2)] = 1
        if s2 in grid.rewards:
          rewards[(s,a,s2)] = grid.rewards[s2]


# fixed policy 
policy = {
    (2,0): 'U',
    (1,0): 'U',
    (0,0): 'R',
    (0,1): 'R',
    (0,2): 'R',
    (1,2): 'U',
    (2,1): 'R',
    (2,2): 'U',
    (2,3): 'L',
}

# Check policy on the grid
print_policy(policy,grid)

# Initialize V(s) = 0
V = {}
for s in grid.all_states():
  V[s] = 0

#print initial V(s)
print('Initial V(s)')
print_values(V,grid)
print('\n')

# Discount Factor
gamma = 0.9

# repeat inult convergenge
it = 0
print('Iteraction Loop \n')
while True:
  biggest_change = 0
  # loop through each state
  for s in grid.all_states():
    # if state = terminal, V(s) is always 0
    if not grid.is_terminal(s):
      old_v = V[s]
      new_v = 0
      # loop through every action for each state
      for a in ACTION_SPACE:
        for s2 in grid.all_states():

          #action probability is deterministic
          action_prob = 1 if policy.get(s) == a else 0

          #reward is a function of (s,a,s'), 0 if not specified
          r = rewards.get((s,a,s2),0)
          # Bellman's Equation
          new_v += action_prob* transition_probs.get((s,a,s2),0) * (r + gamma * V[s2])
      
      #update the value table
      V[s] = new_v
      biggest_change = max(biggest_change, np.abs(old_v - V[s]))

  # monitoring iteraction | change
  print('iteraction:', it, 'biggest change:', biggest_change)
  # monitoring V
  print_values(V,grid)
  print('\n')
  it += 1

  # check convergence
  if biggest_change < conv_threshold:
    break
print('\n\n')



Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  U  |     |
-----------------------------
  U  |  R  |  U  |  L  |


Initial V(s)
-------------------------------
 0.00| 0.00| 0.00| 0.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|


Iteraction Loop 

iteraction: 0 biggest change: 1.0
-------------------------------
 0.00| 0.00| 1.00| 0.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|


iteraction: 1 biggest change: 0.9
-------------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------------
 0.73| 0.00| 0.90| 0.00|
-------------------------------
 0.00| 0.00| 0.81| 0.00|


iteraction: 2 biggest change: 0.7290000000000001
-------------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------------
 0.73| 0.00| 0.90| 0.00|
-------------------------------
 0.66| 0.73

#### Windy Gridworld

In [0]:
#grid world class

class WindyGrid: # Environment
  def __init__ (self, rows, cols, start):
    self.rows = rows
    self.cols = cols
    self.i = start[0]
    self.j = start[1]

  def set(self, rewards, actions,probs):
    # rewards is a dict of: (i,j): r
    # actions is a dict of: (i,j): A
    self.rewards = rewards
    self.actions = actions
    self.probs = probs

  def set_state(self,s):
    #force state
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    #return the current state
    return (self.i, self.j)
  
  def is_terminal(self, s):
    #if state is not listed in the actions dictionary, it means it's a terminal state
    return s not in self.actions

  def move(self, action):
    #current state [s]
    s = (self.i, self.j)
    #action to perform [a]
    a = action

    next_state_probs = self.probs[(s,a)]
    #possible next states
    next_states = list(next_state_probs.keys())
    #probabilities of each next possible state
    next_probs = list(next_state_probs.values())

    #next state [s2]
    s2 = np.random.choice(next_states, p=next_probs)

    #update current state
    self.i, self.j = s2

    #return a reward (if any)
    return self.rewards.get(s2,0)
 
  def game_over(self):
    #true if in a state where no actions are possible
    return (self.i,self.j) not in self.actions

  def all_states(self):
    #either a position that has possible next actions or a positions that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())


In [0]:
def windy_grid():
  g = WindyGrid(3,4, (2,0))
  rewards = {(0,3): 1, (1,3): -1}
  actions = {
      (0,0): ('D','R'),
      (0,1): ('L','R'),
      (0,2): ('L', 'D', 'R'),
      (1,0): ('U', 'D'),
      (1,2): ('U', 'D', 'R'),
      (2,0): ('U', 'R'),
      (2,1): ('L', 'R'),
      (2,2): ('L', 'R', 'U'),
      (2,3): ('L', 'U')
  }

  # p(s' | s, a) represented as:
  # KEY: (s, a) --> VALUE: {s': p(s' | s, a)}
  probs = {
    ((2, 0), 'U'): {(1, 0): 1.0},
    ((2, 0), 'D'): {(2, 0): 1.0},
    ((2, 0), 'L'): {(2, 0): 1.0},
    ((2, 0), 'R'): {(2, 1): 1.0},
    ((1, 0), 'U'): {(0, 0): 1.0},
    ((1, 0), 'D'): {(2, 0): 1.0},
    ((1, 0), 'L'): {(1, 0): 1.0},
    ((1, 0), 'R'): {(1, 0): 1.0},
    ((0, 0), 'U'): {(0, 0): 1.0},
    ((0, 0), 'D'): {(1, 0): 1.0},
    ((0, 0), 'L'): {(0, 0): 1.0},
    ((0, 0), 'R'): {(0, 1): 1.0},
    ((0, 1), 'U'): {(0, 1): 1.0},
    ((0, 1), 'D'): {(0, 1): 1.0},
    ((0, 1), 'L'): {(0, 0): 1.0},
    ((0, 1), 'R'): {(0, 2): 1.0},
    ((0, 2), 'U'): {(0, 2): 1.0},
    ((0, 2), 'D'): {(1, 2): 1.0},
    ((0, 2), 'L'): {(0, 1): 1.0},
    ((0, 2), 'R'): {(0, 3): 1.0},
    ((2, 1), 'U'): {(2, 1): 1.0},
    ((2, 1), 'D'): {(2, 1): 1.0},
    ((2, 1), 'L'): {(2, 0): 1.0},
    ((2, 1), 'R'): {(2, 2): 1.0},
    ((2, 2), 'U'): {(1, 2): 1.0},
    ((2, 2), 'D'): {(2, 2): 1.0},
    ((2, 2), 'L'): {(2, 1): 1.0},
    ((2, 2), 'R'): {(2, 3): 1.0},
    ((2, 3), 'U'): {(1, 3): 1.0},
    ((2, 3), 'D'): {(2, 3): 1.0},
    ((2, 3), 'L'): {(2, 2): 1.0},
    ((2, 3), 'R'): {(2, 3): 1.0},
    ((1, 2), 'U'): {(0, 2): 0.5, (1, 3): 0.5},
    ((1, 2), 'D'): {(2, 2): 1.0},
    ((1, 2), 'L'): {(1, 2): 1.0},
    ((1, 2), 'R'): {(1, 3): 1.0},
  }
  g.set(rewards, actions, probs)
  return g

In [17]:
#main

### define transition probabilities dictionary
# s = current state
# a = action
# s' = next state
# key is (s, a, s'), the value is the probability
# that is: transition_probs[(s,a,s')] = p(s' | s,a)

transition_probs = {}

### define the rewards dictionary
# in order to reduce dimensionality of the dictionary, we'll use deterministic
# rewards, r(s, a, s')
rewards = {}

#define grid world environment
grid = windy_grid()

#populate transition_probs and rewards using the information defined in
# the windy_grid function
for (s,a), v in grid.probs.items():
  for s2, p in v.items():
    transition_probs[(s,a,s2)] = p
    rewards[(s, a, s2)] = grid.rewards.get(s2,0)

#probabilistic policy
policy = {
    (2,0): {'U': 0.5, 'R': 0.5},
    (1,0): {'U': 1.0},
    (0,0): {'R': 1.0},
    (0,1): {'R': 1.0},
    (0,2): {'R': 1.0},
    (1,2): {'U': 1.0},
    (2,1): {'R': 1.0},
    (2,2): {'U': 1.0},
    (2,3): {'L': 1.0},
}

# Check policy on the grid
print_policy(policy,grid)

# Initialize V(s) = 0
V = {}
for s in grid.all_states():
  V[s] = 0

#print initial V(s)
print('Initial V(s)')
print_values(V,grid)
print('\n')

# Discount Factor
gamma = 0.9

# repeat inult convergenge
it = 0
print('Iteraction Loop \n')
while True:
  biggest_change = 0
  # loop through each state
  for s in grid.all_states():
    # if state = terminal, V(s) is always 0
    if not grid.is_terminal(s):
      old_v = V[s]
      new_v = 0
      # loop through every action for each state
      for a in ACTION_SPACE:
        for s2 in grid.all_states():

          #action probability
          action_prob = policy[s].get(a,0)

          #reward is a function of (s,a,s'), 0 if not specified
          r = rewards.get((s,a,s2),0)
          # Bellman's Equation
          new_v += action_prob* transition_probs.get((s,a,s2),0) * (r + gamma * V[s2])
      
      #update the value table
      V[s] = new_v
      biggest_change = max(biggest_change, np.abs(old_v - V[s]))

  # monitoring iteraction | change
  print('iteraction:', it, 'biggest change:', biggest_change)
  # monitoring V
  print_values(V,grid)
  print('\n')
  it += 1

  # check convergence
  if biggest_change < conv_threshold:
    break
print('\n\n')


Policy 

-----------------------------
  {'R': 1.0}  |  {'R': 1.0}  |  {'R': 1.0}  |     |
-----------------------------
  {'U': 1.0}  |     |  {'U': 1.0}  |     |
-----------------------------
  {'U': 0.5, 'R': 0.5}  |  {'R': 1.0}  |  {'U': 1.0}  |  {'L': 1.0}  |


Initial V(s)
-------------------------------
 0.00| 0.00| 0.00| 0.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|


Iteraction Loop 

iteraction: 0 biggest change: 1.0
-------------------------------
 0.00| 0.00| 1.00| 0.00|
-------------------------------
 0.00| 0.00|-0.50| 0.00|
-------------------------------
 0.00| 0.00|-0.45| 0.00|


iteraction: 1 biggest change: 0.9
-------------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------------
 0.73| 0.00|-0.05| 0.00|
-------------------------------
-0.18|-0.41|-0.04|-0.41|


iteraction: 2 biggest change: 0.4920750000000001
-------------------------------
 0.81| 0.90| 1.00| 0.00|
---------

#### Policy Iteraction

In [0]:
# import packages
import numpy as np

In [0]:
### parameters

# convergence threshold
conv_parameter = 1e-3

# Discount Factor
gamma = 0.9

In [0]:
# transition probabilities and rewards

def get_transition_probs_and_rewards(grid):
  ### define transition probabilities dictionary
  # s = current state
  # a = action
  # s' = next state
  # key is (s, a, s'), the value is the probability
  # that is: transition_probs[(s,a,s')] = p(s' | s,a)

  transition_probs = {}

  ### define the rewards dictionary
  # in order to reduce dimensionality of the dictionary, we'll use deterministic
  # rewards, r(s, a, s')
  rewards = {}

  # populate both dictionaries

  for i in range(grid.rows):
    for j in range(grid.cols):
      s = (i,j)
      if not grid.is_terminal(s):
        for a in ACTION_SPACE:
          s2 = grid.get_next_state(s,a)
          transition_probs[(s,a,s2)] = 1
          if s2 in grid.rewards:
            rewards[(s,a,s2)] = grid.rewards[s2]

  return transition_probs, rewards


In [0]:
def evaluate_deterministic_policy(grid,policy):

  #initialize V(s) = 0
  V = {}

  for s in grid.all_states():
    V[s] = 0

  # repeat until convergence
  it = 0
  while True:
    biggest_change = 0
    for s in grid.all_states():
      if not grid.is_terminal(s):
        old_v = V[s]
        new_v = 0 
        for a in ACTION_SPACE:
          for s2 in grid.all_states():

            #action probability is deterministic
            action_prob = 1 if policy.get(s) == a else 0

            #reward is a function of (s,a,s'), 0 if not specified
            r = rewards.get((s,a,s2),0)
            # Bellman's Equation
            new_v += action_prob* transition_probs.get((s,a,s2),0) * (r + gamma * V[s2])
        
        #update the value table
        V[s] = new_v
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    it += 1

    if biggest_change < conv_parameter:
      break

  return V
    


In [97]:
### main
grid = standard_grid()
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("Initial policy: \n")
print_policy(policy, grid)

# repeat until convergence - will break out when policy does not change
while True:

  # policy evaluation step - we already know how to do this!
  V = evaluate_deterministic_policy(grid, policy)

  # policy improvement step
  is_policy_converged = True
  for s in grid.actions.keys():
    old_a = policy[s]
    new_a = None
    best_value = float('-inf')

    # loop through all possible actions to find the best current action
    for a in ACTION_SPACE:
      v = 0
      for s2 in grid.all_states():
        # reward is a function of (s, a, s'), 0 if not specified
        r = rewards.get((s, a, s2), 0)
        v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

      if v > best_value:
        best_value = v
        new_a = a

    # new_a now represents the best action in this state
    policy[s] = new_a
    if new_a != old_a:
      is_policy_converged = False

  if is_policy_converged:
    break

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)


Rewards:
-------------------------------
 0.00| 0.00| 0.00| 1.00|
-------------------------------
 0.00| 0.00| 0.00|-1.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|


Initial policy: 

Policy 

-----------------------------
  U  |  D  |  R  |     |
-----------------------------
  R  |     |  R  |     |
-----------------------------
  L  |  U  |  U  |  D  |


Final Values: 

-------------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------------
 0.73| 0.00| 0.90| 0.00|
-------------------------------
 0.66| 0.73| 0.81| 0.73|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  U  |     |
-----------------------------
  U  |  R  |  U  |  L  |




#### Policy Interaction in the Windy Grid World

In [0]:
# Penalized Windy Grid Environment

def windy_grid_penalized(step_cost = -0.1):
  
  # 3x4 grid, initial state = (2,0)
  g = WindyGrid(3,4, (2,0))

  rewards = {
      (0,0): step_cost,
      (0,1): step_cost,
      (0,2): step_cost,
      (1,0): step_cost,
      (1,2): step_cost,
      (2,0): step_cost,
      (2,1): step_cost,
      (2,2): step_cost,
      (2,3): step_cost,
      (0,3): 1,
      (1,3): -1
  }


  actions = {
      (0,0): ('D','R'),
      (0,1): ('L','R'),
      (0,2): ('L', 'D', 'R'),
      (1,0): ('U', 'D'),
      (1,2): ('U', 'D', 'R'),
      (2,0): ('U', 'R'),
      (2,1): ('L', 'R'),
      (2,2): ('L', 'R', 'U'),
      (2,3): ('L', 'U')
  }

  # p(s' | s, a) represented as:
  # KEY: (s, a) --> VALUE: {s': p(s' | s, a)}
  probs = {
    ((2, 0), 'U'): {(1, 0): 1.0},
    ((2, 0), 'D'): {(2, 0): 1.0},
    ((2, 0), 'L'): {(2, 0): 1.0},
    ((2, 0), 'R'): {(2, 1): 1.0},
    ((1, 0), 'U'): {(0, 0): 1.0},
    ((1, 0), 'D'): {(2, 0): 1.0},
    ((1, 0), 'L'): {(1, 0): 1.0},
    ((1, 0), 'R'): {(1, 0): 1.0},
    ((0, 0), 'U'): {(0, 0): 1.0},
    ((0, 0), 'D'): {(1, 0): 1.0},
    ((0, 0), 'L'): {(0, 0): 1.0},
    ((0, 0), 'R'): {(0, 1): 1.0},
    ((0, 1), 'U'): {(0, 1): 1.0},
    ((0, 1), 'D'): {(0, 1): 1.0},
    ((0, 1), 'L'): {(0, 0): 1.0},
    ((0, 1), 'R'): {(0, 2): 1.0},
    ((0, 2), 'U'): {(0, 2): 1.0},
    ((0, 2), 'D'): {(1, 2): 1.0},
    ((0, 2), 'L'): {(0, 1): 1.0},
    ((0, 2), 'R'): {(0, 3): 1.0},
    ((2, 1), 'U'): {(2, 1): 1.0},
    ((2, 1), 'D'): {(2, 1): 1.0},
    ((2, 1), 'L'): {(2, 0): 1.0},
    ((2, 1), 'R'): {(2, 2): 1.0},
    ((2, 2), 'U'): {(1, 2): 1.0},
    ((2, 2), 'D'): {(2, 2): 1.0},
    ((2, 2), 'L'): {(2, 1): 1.0},
    ((2, 2), 'R'): {(2, 3): 1.0},
    ((2, 3), 'U'): {(1, 3): 1.0},
    ((2, 3), 'D'): {(2, 3): 1.0},
    ((2, 3), 'L'): {(2, 2): 1.0},
    ((2, 3), 'R'): {(2, 3): 1.0},
    ((1, 2), 'U'): {(0, 2): 0.5, (1, 3): 0.5},
    ((1, 2), 'D'): {(2, 2): 1.0},
    ((1, 2), 'L'): {(1, 2): 1.0},
    ((1, 2), 'R'): {(1, 3): 1.0},
  }
  g.set(rewards, actions, probs)
  return g

In [0]:
# transition probabilities and rewards

def get_transition_probs_and_rewards(grid):
  ### define transition probabilities dictionary
  # s = current state
  # a = action
  # s' = next state
  # key is (s, a, s'), the value is the probability
  # that is: transition_probs[(s,a,s')] = p(s' | s,a)

  transition_probs = {}

  ### define the rewards dictionary
  # in order to reduce dimensionality of the dictionary, we'll use deterministic
  # rewards, r(s, a, s')
  rewards = {}

  # populate both dictionaries

  for (s,a), v in grid.probs.items():
    for s2, p in v.items():
      transition_probs[(s,a,s2)] = p
      rewards[(s,a,s2)] = grid.rewards.get(s2,0)

  return transition_probs, rewards

In [0]:
# Evaluate deterministic policy

def evaluate_deterministic_policy(grid,policy):
  #initialize V(s) = 0
  V = {}

  for s in grid.all_states():
    V[s] = 0

  # repeat until convergence
  it = 0
  while True:
    biggest_change = 0
    for s in grid.all_states():
      if not grid.is_terminal(s):
        old_v = V[s]
        new_v = 0 
        for a in ACTION_SPACE:
          for s2 in grid.all_states():

            #action probability is deterministic
            action_prob = 1 if policy.get(s) == a else 0

            #reward is a function of (s,a,s'), 0 if not specified
            r = rewards.get((s,a,s2),0)
            # Bellman's Equation
            new_v += action_prob* transition_probs.get((s,a,s2),0) * (r + gamma * V[s2])
        
        #update the value table
        V[s] = new_v
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    it += 1

    if biggest_change < conv_parameter:
      break

  return V

**Check Regular Windy Grid**

In [105]:
### main

grid = windy_grid()
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("Initial policy: \n")
print_policy(policy, grid)

# repeat until convergence - will break out when policy does not change
while True:

  # policy evaluation step - we already know how to do this!
  V = evaluate_deterministic_policy(grid, policy)

  # policy improvement step
  is_policy_converged = True
  for s in grid.actions.keys():
    old_a = policy[s]
    new_a = None
    best_value = float('-inf')

    # loop through all possible actions to find the best current action
    for a in ACTION_SPACE:
      v = 0
      for s2 in grid.all_states():
        # reward is a function of (s, a, s'), 0 if not specified
        r = rewards.get((s, a, s2), 0)
        v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

      if v > best_value:
        best_value = v
        new_a = a

    # new_a now represents the best action in this state
    policy[s] = new_a
    if new_a != old_a:
      is_policy_converged = False

  if is_policy_converged:
    break

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)


Rewards:
-------------------------------
 0.00| 0.00| 0.00| 1.00|
-------------------------------
 0.00| 0.00| 0.00|-1.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|


Initial policy: 

Policy 

-----------------------------
  D  |  R  |  R  |     |
-----------------------------
  L  |     |  R  |     |
-----------------------------
  U  |  L  |  L  |  D  |


Final Values: 

-------------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------------
 0.73| 0.00| 0.48| 0.00|
-------------------------------
 0.66| 0.59| 0.53| 0.48|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  D  |     |
-----------------------------
  U  |  L  |  L  |  L  |




**Check Penalized Windy Grid**

In [106]:
grid = windy_grid_penalized()
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("Initial policy: \n")
print_policy(policy, grid)

# repeat until convergence - will break out when policy does not change
while True:

  # policy evaluation step - we already know how to do this!
  V = evaluate_deterministic_policy(grid, policy)

  # policy improvement step
  is_policy_converged = True
  for s in grid.actions.keys():
    old_a = policy[s]
    new_a = None
    best_value = float('-inf')

    # loop through all possible actions to find the best current action
    for a in ACTION_SPACE:
      v = 0
      for s2 in grid.all_states():
        # reward is a function of (s, a, s'), 0 if not specified
        r = rewards.get((s, a, s2), 0)
        v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

      if v > best_value:
        best_value = v
        new_a = a

    # new_a now represents the best action in this state
    policy[s] = new_a
    if new_a != old_a:
      is_policy_converged = False

  if is_policy_converged:
    break

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)

Rewards:
-------------------------------
-0.10|-0.10|-0.10| 1.00|
-------------------------------
-0.10| 0.00|-0.10|-1.00|
-------------------------------
-0.10|-0.10|-0.10|-0.10|


Initial policy: 

Policy 

-----------------------------
  R  |  D  |  R  |     |
-----------------------------
  U  |     |  D  |     |
-----------------------------
  L  |  L  |  D  |  R  |


Final Values: 

-------------------------------
 0.62| 0.80| 1.00| 0.00|
-------------------------------
 0.46| 0.00|-0.04| 0.00|
-------------------------------
 0.31| 0.18| 0.06|-0.04|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  D  |     |
-----------------------------
  U  |  L  |  L  |  L  |




In [108]:
# increase penalty (-0.2)
grid = windy_grid_penalized(-0.2)
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("Initial policy: \n")
print_policy(policy, grid)

# repeat until convergence - will break out when policy does not change
while True:

  # policy evaluation step - we already know how to do this!
  V = evaluate_deterministic_policy(grid, policy)

  # policy improvement step
  is_policy_converged = True
  for s in grid.actions.keys():
    old_a = policy[s]
    new_a = None
    best_value = float('-inf')

    # loop through all possible actions to find the best current action
    for a in ACTION_SPACE:
      v = 0
      for s2 in grid.all_states():
        # reward is a function of (s, a, s'), 0 if not specified
        r = rewards.get((s, a, s2), 0)
        v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

      if v > best_value:
        best_value = v
        new_a = a

    # new_a now represents the best action in this state
    policy[s] = new_a
    if new_a != old_a:
      is_policy_converged = False

  if is_policy_converged:
    break

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)

Rewards:
-------------------------------
-0.20|-0.20|-0.20| 1.00|
-------------------------------
-0.20| 0.00|-0.20|-1.00|
-------------------------------
-0.20|-0.20|-0.20|-0.20|


Initial policy: 

Policy 

-----------------------------
  R  |  L  |  R  |     |
-----------------------------
  U  |     |  U  |     |
-----------------------------
  D  |  L  |  D  |  D  |


Final Values: 

-------------------------------
 0.43| 0.70| 1.00| 0.00|
-------------------------------
 0.19| 0.00|-0.15| 0.00|
-------------------------------
-0.03|-0.23|-0.34|-0.50|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  U  |     |
-----------------------------
  U  |  L  |  U  |  L  |




In [109]:
# increase penalty (-0.4)
grid = windy_grid_penalized(-0.4)
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("Initial policy: \n")
print_policy(policy, grid)

# repeat until convergence - will break out when policy does not change
while True:

  # policy evaluation step - we already know how to do this!
  V = evaluate_deterministic_policy(grid, policy)

  # policy improvement step
  is_policy_converged = True
  for s in grid.actions.keys():
    old_a = policy[s]
    new_a = None
    best_value = float('-inf')

    # loop through all possible actions to find the best current action
    for a in ACTION_SPACE:
      v = 0
      for s2 in grid.all_states():
        # reward is a function of (s, a, s'), 0 if not specified
        r = rewards.get((s, a, s2), 0)
        v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

      if v > best_value:
        best_value = v
        new_a = a

    # new_a now represents the best action in this state
    policy[s] = new_a
    if new_a != old_a:
      is_policy_converged = False

  if is_policy_converged:
    break

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)

Rewards:
-------------------------------
-0.40|-0.40|-0.40| 1.00|
-------------------------------
-0.40| 0.00|-0.40|-1.00|
-------------------------------
-0.40|-0.40|-0.40|-0.40|


Initial policy: 

Policy 

-----------------------------
  U  |  R  |  L  |     |
-----------------------------
  R  |     |  D  |     |
-----------------------------
  D  |  D  |  L  |  U  |


Final Values: 

-------------------------------
 0.05| 0.50| 1.00| 0.00|
-------------------------------
-0.36| 0.00|-0.25| 0.00|
-------------------------------
-0.72|-0.96|-0.62|-0.96|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  U  |     |
-----------------------------
  U  |  R  |  U  |  L  |




In [110]:
# increase penalty (-0.5)
grid = windy_grid_penalized(-0.5)
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("Initial policy: \n")
print_policy(policy, grid)

# repeat until convergence - will break out when policy does not change
while True:

  # policy evaluation step - we already know how to do this!
  V = evaluate_deterministic_policy(grid, policy)

  # policy improvement step
  is_policy_converged = True
  for s in grid.actions.keys():
    old_a = policy[s]
    new_a = None
    best_value = float('-inf')

    # loop through all possible actions to find the best current action
    for a in ACTION_SPACE:
      v = 0
      for s2 in grid.all_states():
        # reward is a function of (s, a, s'), 0 if not specified
        r = rewards.get((s, a, s2), 0)
        v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

      if v > best_value:
        best_value = v
        new_a = a

    # new_a now represents the best action in this state
    policy[s] = new_a
    if new_a != old_a:
      is_policy_converged = False

  if is_policy_converged:
    break

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)

Rewards:
-------------------------------
-0.50|-0.50|-0.50| 1.00|
-------------------------------
-0.50| 0.00|-0.50|-1.00|
-------------------------------
-0.50|-0.50|-0.50|-0.50|


Initial policy: 

Policy 

-----------------------------
  D  |  L  |  U  |     |
-----------------------------
  U  |     |  U  |     |
-----------------------------
  D  |  R  |  U  |  D  |


Final Values: 

-------------------------------
-0.14| 0.40| 1.00| 0.00|
-------------------------------
-0.63| 0.00|-0.30| 0.00|
-------------------------------
-1.06|-1.19|-0.77|-1.00|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  U  |     |
-----------------------------
  U  |  R  |  U  |  U  |




#### Value Iteraction

In [117]:
### Value Optimization

grid = windy_grid()
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# Initialize V(s) = 0
V = {}
states = grid.all_states()
for s in states:
  V[s] = 0

#repeat until convergence
# V[s] = max[a] {sum [s',r] {p(s',r | s,a)*[r + gamma*V[s']]}}
it = 0
while True:
  biggest_change = 0
  #iterate through all states
  for s in grid.all_states():
    # if state is terminal, V[s] is always zero
    if not grid.is_terminal(s):
      old_v = V[s]
      new_v = float('-inf')

      #iterate through all possible actions
      for a in ACTION_SPACE:
        v = 0
        # iterate through all possible next states
        for s2 in grid.all_states():

          #reward is a function of (s,a,s'), 0 if not specified
          r = rewards.get((s,a,s2),0)
          
          #bellman equation
          v += transition_probs.get((s,a,s2),0) * (r + gamma*V[s2])

        #keep v if it's better
        if v > new_v:
          new_v = v

      V[s] = new_v
      biggest_change = max(biggest_change, np.abs(old_v - V[s]))

  it += 1
  if biggest_change < conv_threshold:
    break

# Find a policy that leads to optimal value function

policy = {}
for s in grid.actions.keys():
  best_a = None
  best_value = float('-inf')

  #loop through all possible action to find the best current action
  for a in ACTION_SPACE:
    v = 0
    for s2 in grid.all_states():

      #reward is a function of (s,a,s'), 0 if not specified
      r = rewards.get((s,a,s2),0)

      #bellman equation
      v += transition_probs.get((s,a,s2),0) * (r + gamma*V[s2])

    #best_a is the action associated with best_value
    if v > best_value:
      best_value = v
      best_a = a
  
  policy[s] = best_a

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)


Rewards:
-------------------------------
 0.00| 0.00| 0.00| 1.00|
-------------------------------
 0.00| 0.00| 0.00|-1.00|
-------------------------------
 0.00| 0.00| 0.00| 0.00|


Final Values: 

-------------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------------
 0.73| 0.00| 0.48| 0.00|
-------------------------------
 0.66| 0.59| 0.53| 0.48|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  D  |     |
-----------------------------
  U  |  L  |  L  |  L  |




In [118]:
### Value Optimization

grid = windy_grid_penalized()
transition_probs, rewards = get_transition_probs_and_rewards(grid)

# print rewards
print("Rewards:")
print_values(grid.rewards, grid)
print('\n')

# Initialize V(s) = 0
V = {}
states = grid.all_states()
for s in states:
  V[s] = 0

#repeat until convergence
# V[s] = max[a] {sum [s',r] {p(s',r | s,a)*[r + gamma*V[s']]}}
it = 0
while True:
  biggest_change = 0
  #iterate through all states
  for s in grid.all_states():
    # if state is terminal, V[s] is always zero
    if not grid.is_terminal(s):
      old_v = V[s]
      new_v = float('-inf')

      #iterate through all possible actions
      for a in ACTION_SPACE:
        v = 0
        # iterate through all possible next states
        for s2 in grid.all_states():

          #reward is a function of (s,a,s'), 0 if not specified
          r = rewards.get((s,a,s2),0)
          
          #bellman equation
          v += transition_probs.get((s,a,s2),0) * (r + gamma*V[s2])

        #keep v if it's better
        if v > new_v:
          new_v = v

      V[s] = new_v
      biggest_change = max(biggest_change, np.abs(old_v - V[s]))

  it += 1
  if biggest_change < conv_threshold:
    break

# Find a policy that leads to optimal value function

policy = {}
for s in grid.actions.keys():
  best_a = None
  best_value = float('-inf')

  #loop through all possible action to find the best current action
  for a in ACTION_SPACE:
    v = 0
    for s2 in grid.all_states():

      #reward is a function of (s,a,s'), 0 if not specified
      r = rewards.get((s,a,s2),0)

      #bellman equation
      v += transition_probs.get((s,a,s2),0) * (r + gamma*V[s2])

    #best_a is the action associated with best_value
    if v > best_value:
      best_value = v
      best_a = a
  
  policy[s] = best_a

# once we're done, print the final policy and values
print("Final Values: \n")
print_values(V, grid)
print('\n')
print("Final Policy: \n")
print_policy(policy, grid)


Rewards:
-------------------------------
-0.10|-0.10|-0.10| 1.00|
-------------------------------
-0.10| 0.00|-0.10|-1.00|
-------------------------------
-0.10|-0.10|-0.10|-0.10|


Final Values: 

-------------------------------
 0.62| 0.80| 1.00| 0.00|
-------------------------------
 0.46| 0.00|-0.04| 0.00|
-------------------------------
 0.31| 0.18| 0.06|-0.04|


Final Policy: 

Policy 

-----------------------------
  R  |  R  |  R  |     |
-----------------------------
  U  |     |  D  |     |
-----------------------------
  U  |  L  |  L  |  L  |


